In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax

In [2]:
train_activations = np.load('/Users/alfioleanza/progetto_tesi/dataset-eeg/inference_20250327_171717/train_activations.npy', allow_pickle = True,).item()
test_activations = np.load('/Users/alfioleanza/progetto_tesi/dataset-eeg/inference_20250327_171717/test_activations.npy', allow_pickle = True,).item()
val_activations = np.load('/Users/alfioleanza/progetto_tesi/dataset-eeg/inference_20250327_171717/val_activations.npy', allow_pickle = True,).item()


In [3]:
for dataset_name, dataset in zip(['train_activations', 'test_activations', 'val_activations'], [train_activations, test_activations, val_activations]):
    list_of_data = [{'crop_file': key, 'valore': value} for key, value in dataset.items()]
    df = pd.DataFrame(list_of_data)
    globals()[f'{dataset_name}_df'] = df

In [4]:
train_activations_df['dataset'] = 'training'
test_activations_df['dataset'] = 'test'
val_activations_df['dataset'] = 'validation'

In [5]:
for dataset_name, dataset in zip(['train_activations_df', 'test_activations_df', 'val_activations_df'], [train_activations_df, test_activations_df, val_activations_df]):
    globals()[f'{dataset_name}']['valore_softmax'] = globals()[f'{dataset_name}']['valore'].apply(lambda x: softmax(x))
    globals()[f'{dataset_name}']['pred_label'] = globals()[f'{dataset_name}']['valore_softmax'].apply(lambda x: np.argmax(x))

In [6]:
all_activations_df = pd.concat([train_activations_df,test_activations_df,val_activations_df], ignore_index = True)

In [7]:
all_activations_df

,crop_file,valore,dataset,valore_softmax,pred_label
0,00000848.npy,"[1.0532185, -1.5921667, -0.8496607]",training,"[0.8195938, 0.058173135, 0.12223307]",0
1,00000849.npy,"[0.8592405, -0.8918216, -0.32704002]",training,"[0.67615795, 0.1173739, 0.20646815]",0
2,00000850.npy,"[0.89024836, -1.5663235, -0.599953]",training,"[0.7627442, 0.06538879, 0.17186707]",0
3,00000851.npy,"[0.9642988, -1.5698762, -0.83838975]",training,"[0.80374086, 0.06375847, 0.13250075]",0
4,00000852.npy,"[0.9176249, -1.4399661, -0.5990748]",training,"[0.76098704, 0.072025865, 0.1669871]",0
...,...,...,...,...,...
69789,00069084.npy,"[-3.1664958, -1.7627978, -0.22231679]",validation,"[0.04155363, 0.16913258, 0.78931385]",2
69790,00069085.npy,"[-1.5803745, -0.6570947, -0.067650065]",validation,"[0.12412163, 0.3124804, 0.563398]",2
69791,00069086.npy,"[-0.6699866, -0.1551984, -0.40366274]",validation,"[0.2513547, 0.4205878, 0.3280575]",1
69792,00069087.npy,"[-0.6042401, -0.30352032, -0.33559638]",validation,"[0.27329725, 0.36917832, 0.35752445]",1


In [8]:
annot = pd.read_csv('/Users/alfioleanza/progetto_tesi/dataset-eeg/miltiadous_deriv_uV_d1.0s_o0.0s/annot_all_hc-ftd-ad.csv')

In [9]:
annot

,crop_file,label,original_rec,crop_start_sample,crop_end_sample
0,00000000.npy,0,sub-064,0,499.0
1,00000001.npy,0,sub-064,500,999.0
2,00000002.npy,0,sub-064,1000,1499.0
3,00000003.npy,0,sub-064,1500,1999.0
4,00000004.npy,0,sub-064,2000,2499.0
...,...,...,...,...,...
69789,00069789.npy,2,sub-033,350000,350499.0
69790,00069790.npy,2,sub-033,350500,350999.0
69791,00069791.npy,2,sub-033,351000,351499.0
69792,00069792.npy,2,sub-033,351500,351999.0


In [10]:
annot = annot.rename(columns={'label':'true_label'})

In [11]:
true_pred = all_activations_df.merge(annot, on = 'crop_file')

In [12]:
true_pred

,crop_file,valore,dataset,valore_softmax,pred_label,true_label,original_rec,crop_start_sample,crop_end_sample
0,00000848.npy,"[1.0532185, -1.5921667, -0.8496607]",training,"[0.8195938, 0.058173135, 0.12223307]",0,0,sub-037,0,499.0
1,00000849.npy,"[0.8592405, -0.8918216, -0.32704002]",training,"[0.67615795, 0.1173739, 0.20646815]",0,0,sub-037,500,999.0
2,00000850.npy,"[0.89024836, -1.5663235, -0.599953]",training,"[0.7627442, 0.06538879, 0.17186707]",0,0,sub-037,1000,1499.0
3,00000851.npy,"[0.9642988, -1.5698762, -0.83838975]",training,"[0.80374086, 0.06375847, 0.13250075]",0,0,sub-037,1500,1999.0
4,00000852.npy,"[0.9176249, -1.4399661, -0.5990748]",training,"[0.76098704, 0.072025865, 0.1669871]",0,0,sub-037,2000,2499.0
...,...,...,...,...,...,...,...,...,...
69789,00069084.npy,"[-3.1664958, -1.7627978, -0.22231679]",validation,"[0.04155363, 0.16913258, 0.78931385]",2,2,sub-026,446500,446999.0
69790,00069085.npy,"[-1.5803745, -0.6570947, -0.067650065]",validation,"[0.12412163, 0.3124804, 0.563398]",2,2,sub-026,447000,447499.0
69791,00069086.npy,"[-0.6699866, -0.1551984, -0.40366274]",validation,"[0.2513547, 0.4205878, 0.3280575]",1,2,sub-026,447500,447999.0
69792,00069087.npy,"[-0.6042401, -0.30352032, -0.33559638]",validation,"[0.27329725, 0.36917832, 0.35752445]",1,2,sub-026,448000,448499.0


In [13]:
(true_pred['pred_label'] == true_pred['true_label']).value_counts()

True     41839
False    27955
Name: count, dtype: int64

In [14]:
true_pred = true_pred.rename(columns={'valore':'activation_values'})
true_pred = true_pred.rename(columns={'valore_softmax':'softmax_values'})

In [15]:
true_pred.to_csv('/Users/alfioleanza/progetto_tesi/dataset-eeg/inference_20250327_171717/true_pred.csv', index = False)